# LightGCN - Cornac
requirements : torch==2.0.0, dgl==1.1.0,numpy<2.0

In [3]:
import dgl

In [7]:
import cornac
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.models.lightgcn.recom_lightgcn import LightGCN
from cornac.metrics import Precision, Recall, NDCG, MAP

print(f"Cornac version: {cornac.__version__}")

# 1. 데이터셋 로드 (MovieLens 10M)
# 처음 실행 시 데이터를 다운로드하므로 시간이 걸릴 수 있습니다.
print("Loading MovieLens 10M dataset...")
ml_10m = movielens.load_feedback(variant="100k")

# 2. 데이터 분할 및 전처리
# RatioSplit: 데이터를 학습용, 검증용, 테스트용으로 분할합니다.
# - rating_threshold=4.0: 평점 4.0 이상만 '긍정적' 상호작용(implicit feedback)으로 간주합니다.
# - test_size=0.2: 20%를 테스트 데이터로 사용합니다.
# - val_size=0.1: 10%를 검증 데이터로 사용합니다. (나머지 70%는 학습용)
print("Splitting data into training, validation, and test sets...")
ratio_split = RatioSplit(
    data=ml_10m,
    test_size=0.2,
    val_size=0.1,
    rating_threshold=4.0,  # 4점 이상을 긍정적 피드백으로 간주
    seed=123,
    verbose=True
)

# 3. 모델 정의 (LightGCN)
# LightGCN 모델의 하이퍼파라미터를 설정합니다.
# - k: 임베딩 벡터의 차원 (잠재 요인 수)
# - n_epochs: 학습 반복 횟수
# - n_layers: LightGCN의 레이어 수 (그래프 상에서 이웃 정보를 몇 홉까지 전파할지 결정)
# - lambda_reg: 정규화 파라미터 (과적합 방지)
print("Instantiating LightGCN model...")
lightgcn = LightGCN(
    emb_size=64,
    num_epochs=30,
    num_layers=3,
    lambda_reg=1e-4,
    learning_rate=0.001,
    seed=456,
    verbose=True
)

# 4. 평가 지표 정의
# Top-10 추천에 대한 성능을 측정합니다.
metrics = [
    Precision(k=10),
    Recall(k=10),
    NDCG(k=10),
    MAP()
]

# 5. 실험 구성 및 실행
# Cornac의 Experiment는 데이터, 모델, 평가 지표를 하나로 묶어 실험을 자동화합니다.
print("Setting up the experiment...")
experiment = cornac.Experiment(
    eval_method=ratio_split,
    models=[lightgcn],
    metrics=metrics,
    user_based=True  # 사용자 기반 평가 수행
)

print("Running the experiment... (This might take a while for MovieLens 10M)")
experiment.run()

print("Experiment finished.")

Cornac version: 2.2.2
Loading MovieLens 10M dataset...
Splitting data into training, validation, and test sets...
rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1635
Number of ratings = 70000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1635
Number of ratings = 19962
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1635
Number of ratings = 9977
---
Total users = 943
Total items = 1635
Instantiating LightGCN model...
Setting up the experiment...
Running the experiment... (This might take a while for MovieLens 10M)

[LightGCN] Training started!



[LightGCN] Evaluation started!


Ranking: 100%|██████████| 917/917 [00:00<00:00, 968.28it/s]


VALIDATION:
...
         |    MAP | NDCG@10 | Precision@10 | Recall@10 | Time (s)
-------- + ------ + ------- + ------------ + --------- + --------
LightGCN | 0.1014 |  0.1225 |       0.0694 |    0.1395 |   0.9496

TEST:
...
         |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------- + ------ + ------- + ------------ + --------- + --------- + --------
LightGCN | 0.1364 |  0.1890 |       0.1434 |    0.1484 |  300.2270 |   1.2089

Experiment finished.
